Given a string s and a string array dictionary, return the longest string in the dictionary that can be formed by deleting some of the given string characters. If there is more than one possible result, return the longest word with the smallest lexicographical order. If there is no possible result, return the empty string.

 

Example 1:

Input: s = "abpcplea", dictionary = ["ale","apple","monkey","plea"]
Output: "apple"
Example 2:

Input: s = "abpcplea", dictionary = ["a","b","c"]
Output: "a"
 

Constraints:

1 <= s.length <= 1000
1 <= dictionary.length <= 1000
1 <= dictionary[i].length <= 1000
s and dictionary[i] consist of lowercase English letters.

# for brute force
- Sort the dict in the decreasing order, and the word lexicographical order
- then for each word check its sub-sequence of the string or not.
Sorting: O(n log n) where n = len(dictionary)
Subsequence check: O(n × m) (m = len(s))
Total: O(n log n + n × m)
sc - O(1)

In [1]:
from typing import List

class Solution:
    def findLongestWord(self, s: str, dictionary: List[str]) -> str:
        # Helper to check if word is a subsequence of s
        def is_subsequence(word: str, s: str) -> bool:
            i = j = 0
            while i < len(word) and j < len(s):
                if word[i] == s[j]:
                    i += 1
                j += 1
            return i == len(word)

        # Sort dictionary:
        #   1) By length descending
        #   2) Lexicographically ascending
        dictionary.sort(key=lambda w: (-len(w), w))

        for word in dictionary:
            if is_subsequence(word, s):
                return word

        return ""



Instead of checking every word with a two-pointer scan, we **preprocess `s`** into an index map so subsequence checks become **logarithmic**.

---

### 🔹 Idea

1. Preprocess `s` → `char → sorted list of indices`
   Example: `s = "abpcplea"` →

   ```
   {
     'a': [0, 7],
     'b': [1],
     'c': [3],
     'e': [6],
     'l': [5],
     'p': [2, 4]
   }
   ```
2. For each word in dictionary:

   * For each char, binary search for its next position **greater than the last chosen index**.
   * If all chars found → word is subsequence.
3. Return the longest (lexicographically smallest if tie).

---



### 🔹 Complexity

* Preprocessing `s`: `O(m)`
* Each subsequence check: `O(L log m)` (`L = word length`)
* For `n` dictionary words: **`O(m + (n × L log m))`**
* Space: `O(m)` for index map.

This is **much faster** than the two-pointer `O(n × m)` solution when `s` is long and dictionary has many words.



In [ ]:
from typing import List
import bisect

class Solution:
    def findLongestWord(self, s: str, dictionary: List[str]) -> str:
        # Step 1: Preprocess s into {char: indices}
        pos_map = {}
        for i, ch in enumerate(s):
            if ch not in pos_map:
                pos_map[ch] = []
            pos_map[ch].append(i)

        # Helper to check if word is subsequence using binary search
        def is_subsequence(word: str) -> bool:
            prev_index = -1
            for ch in word:
                if ch not in pos_map:
                    return False
                idx_list = pos_map[ch]
                # find smallest index > prev_index
                # It performs binary search on a sorted list.
                # bisect_right(a, x) → returns the index where x should be inserted to keep the list sorted, placing it to the right of any existing x.
                k = bisect.bisect_right(idx_list, prev_index)
                if k == len(idx_list):
                    return False
                prev_index = idx_list[k]
            return True

        # Step 2: Check dictionary words
        best = ""
        for word in dictionary:
            if (len(word) > len(best)) or (len(word) == len(best) and word < best):
                if is_subsequence(word):
                    best = word

        return best
